In [1]:
from tqdm import tqdm
import time
import json
from enum import Enum
from gurobipy import Model, GRB, quicksum, max_
import numpy as np
import pandas as pd
from random import randint
import plotly.graph_objs as go


In [2]:
class DATASET(Enum):
    TOY = "toy"
    MEDIUM = "medium"
    LARGE = "large"

# Fonctions pour préparer le modèle


In [3]:
def load_data(name):
    """name must be an instance of DATASET like DATASET.TOY for example"""
    if not isinstance(name, DATASET):
        raise TypeError("direction must be an instance of DATASET Enum")
    with open(f"../data/{name.value}_instance.json", "r") as f:
        data = json.load(f)
    return data


def get_dims(data):
    return (
        len(data["staff"]),
        data["horizon"],
        len(data["qualifications"]),
        len(data["jobs"]),
    )

def get_qualification_index(list_qualifications, qualification): # qualification is "A", "B", "C" ...
    return list_qualifications.index(qualification)

In [4]:
def init_model():
    m = Model("Project modelling")
    return m

In [5]:
def create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs):
    X = model.addVars(n_staff, horizon, n_qualifs, n_jobs, vtype=GRB.BINARY, name="assignements")
    J = model.addVars(n_jobs, vtype=GRB.BINARY, name="completion")
    E_D = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="end_dates")
    L = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="n_days_late")
    # profit 
    profit = model.addVar(lb =0, vtype = GRB.INTEGER, name = "profit")
    # for max(days spent on project) variable
    S_D = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="start_dates")
    spans = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="spans")
    max_days = model.addVar(lb=0, ub=horizon, vtype=GRB.INTEGER, name="max_jobs")
    # for max(projects affected) variable 
    n_jobs_per_person = model.addVars(n_staff, lb=0, ub=n_jobs, vtype=GRB.INTEGER, name="n_jobs_per_person")
    jobs_worked_on_by_person = model.addVars(n_staff, n_jobs, vtype=GRB.BINARY, name="jobs_worked_on_by_person")
    n_worked_days_per_job_and_person = model.addVars(n_staff, n_jobs,lb=0, ub=horizon, vtype=GRB.INTEGER, name="n_worked_days_per_job_and_person")
    max_jobs = model.addVar(lb=0, ub=n_jobs, vtype=GRB.INTEGER, name="max_jobs")
    return model, X, J, S_D, E_D, L, n_jobs_per_person, jobs_worked_on_by_person, n_worked_days_per_job_and_person, max_jobs, max_days, spans, profit


In [6]:
def add_constraints_start_dates(model, X, S_D, n_staff, horizon, n_qualifs, n_jobs):
    # model.addConstrs(
    #     (X[i, j, k, l] == 1) >> (X[i, j, k, l] * j >= S_D[l]) 
    #     for i in range(n_staff)
    #     for j in range(horizon)
    #     for k in range(n_qualifs)
    #     for l in range(n_jobs)
    # )
    model.addConstrs(
        S_D[l] <= X[i, j, k, l] * j + (1 - X[i, j, k, l]) * horizon
        for i in range(n_staff)
        for j in range(horizon) 
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [7]:
def add_constraints_end_dates(model, X, E_D, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        X[i, j, k, l] * j <= E_D[l] 
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [8]:
def add_constraints_lateness(model, E_D, L, jobs, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        E_D[l] - jobs[l]["due_date"] <= L[l] for l in range(n_jobs)
    )

In [9]:
def add_constraints_worked_days_below_required_days(model, X, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) <= jobs[l]["working_days_per_qualification"][k] 
                     for l in range(n_jobs) 
                     for k in jobs[l]["working_days_per_qualification"].keys())
    

In [10]:
def add_constraints_worked_days_above_required_days(model, X, J, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) >= J[l]* jobs[l]["working_days_per_qualification"][k] 
                     for l in range(n_jobs) 
                     for k in jobs[l]["working_days_per_qualification"].keys())

    # for l in range(n_jobs): 
    #     for quali,njk in data["jobs"][l]["working_days_per_qualification"].items():
    #         model.addConstr(J[l]*njk <= quicksum([X[i,j,get_qualification_index(qualifications, quali),l] for i in range(n_staff) for j in range(horizon)]))

In [11]:
def add_constraints_employees_working_only_one_day(model, X, J, data, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs(quicksum(X[i,j,k,l] for l in range(n_jobs) for k in range (n_qualifs)) <= 1 
                     for i in range(n_staff) 
                     for j in range(horizon))

In [12]:
def in_qualification(data, i, k):
    return data["qualifications"][k] in data["staff"][i]["qualifications"]


def add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if not in_qualification(data, i, k)
    )


def in_vacation(i, j, data):
    data = {l: data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["vacations"]
    return j in data


def add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if in_vacation(i, j, data)
    )

In [13]:
def add_constraint_profit(model, J, L, profit, jobs):
    model.addConstr(profit == quicksum( (J[index_job] * job["gain"] - job["daily_penalty"] * L[index_job]) for index_job, job in enumerate(jobs)))

In [14]:
def add_constraints_n_worked_days_per_jobs_person(model, X, n_worked_days_per_job_and_person, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs( n_worked_days_per_job_and_person[i, l] == quicksum( X[i,j,k,l] for j in range(horizon) for k in range(n_qualifs)) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    )

def add_constraints_jobs_worked_on_by_person(model, jobs_worked_on_by_person, n_worked_days_per_job_and_person, n_staff, n_jobs):
    model.addConstrs((jobs_worked_on_by_person[i, l] == 0) >> (n_worked_days_per_job_and_person[i,l] == 0) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    ) 
    # model.addConstrs((jobs_worked_on_by_person[i, l] <= n_worked_days_per_job_and_person[i,l]) 
    #     for i in range(n_staff) 
    #     for l in range(n_jobs)
    # ) 
    model.addConstrs((jobs_worked_on_by_person[i, l] == 1) >> (n_worked_days_per_job_and_person[i,l] >= 1) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    )
    # model.addConstrs((n_worked_days_per_job_and_person[i, l] <= jobs_worked_on_by_person[i,l]*n_jobs) 
    #     for i in range(n_staff) 
    #     for l in range(n_jobs)
    # ) 

def add_constraints_n_jobs_per_person(model, n_jobs_per_person, jobs_worked_on_by_person, n_staff, n_jobs):
    model.addConstrs(n_jobs_per_person[i] == quicksum( jobs_worked_on_by_person[i, l] for l in range(n_jobs) ) for i in range(n_staff))

def add_constraint_max_jobs(model, max_jobs, n_jobs_per_person, jobs_worked_on_by_person, n_staff):
    model.addConstr(max_jobs == max_([n_jobs_per_person[i] for i in range(n_staff)]))

In [15]:
def add_constraint_spans(model, spans, E_D, S_D, n_jobs):
    model.addConstrs((spans[l] == (E_D[l] - S_D[l])+1) for l in range(n_jobs))

def add_constraint_max_days(model, max_days, spans, n_jobs):
    for l in range(n_jobs):
        model.addConstr(max_days >= spans[l]) 
    # model.addConstr(max_days == max_([spans[l] for l in range(n_jobs)]))

In [16]:
def add_profit_as_first_objective(model, J, L, jobs):
    model.setObjective(
        quicksum( (J[index_job] * job["gain"] - job["daily_penalty"] * L[index_job]) for index_job, job in enumerate(jobs)),
        GRB.MAXIMIZE
    )
    return model

## add pouieme des autres

In [17]:
def add_minimax_jobs_as_second_objective(model, max_jobs):
    model.setObjective(
        max_jobs,
        GRB.MINIMIZE
    )
    return model

In [18]:
def add_minimax_days_spent_as_third_objective(model, max_days):
    model.setObjective(
        max_days,
        GRB.MINIMIZE
    )
    return model

In [19]:
def add_mono_objective(model, profit, max_days, max_jobs):
    model.setObjective(
        10 * profit - max_days - max_jobs,
        GRB.MAXIMIZE
    )
    return model, profit, max_days, max_jobs

# Surface

In [20]:
def prepare_model(data):
    n_staff, horizon, n_qualifs, n_jobs = get_dims(data)

    # Instanciation du modèle
    model = init_model()
    
    # Création des variables : binaires dans X et J, entières de 0 à horizon + 3
    model, X, J, S_D, E_D, L, n_jobs_per_person, jobs_worked_on_by_person, n_worked_days_per_job_and_person, max_jobs, max_days, spans, profit = create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs)
    
    # maj du modèle
    model.update()
    
    # Ajout des constraintes
    
    add_constraints_employees_working_only_one_day(model, X ,J,data,n_staff,n_jobs,horizon, n_qualifs)
    add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    
    add_constraints_start_dates(model, X, S_D, n_staff, horizon, n_qualifs, n_jobs)
    add_constraints_end_dates(model, X, E_D, n_staff, horizon, n_qualifs, n_jobs)
    add_constraints_lateness(model, E_D, L, data["jobs"], n_staff, horizon, n_qualifs, n_jobs)
    
    add_constraints_worked_days_below_required_days(model, X, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
    add_constraints_worked_days_above_required_days(model, X , J, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
    
    add_constraints_n_worked_days_per_jobs_person(model, X, n_worked_days_per_job_and_person, n_staff, n_jobs, horizon, n_qualifs)
    add_constraints_jobs_worked_on_by_person(model, jobs_worked_on_by_person, n_worked_days_per_job_and_person, n_staff, n_jobs)
    add_constraints_n_jobs_per_person(model, n_jobs_per_person, jobs_worked_on_by_person, n_staff, n_jobs)
    add_constraint_spans(model, spans, E_D, S_D, n_jobs)
    
    add_constraint_profit(model, J, L, profit, data["jobs"])
    add_constraint_max_jobs(model, max_jobs, n_jobs_per_person, jobs_worked_on_by_person, n_staff)
    add_constraint_max_days(model, max_days, spans, n_jobs)
    
    
    # Fonctions Objectifs
    
    # model = add_minimax_jobs_as_second_objective(model, max_jobs)
    # model = add_minimax_days_spent_as_third_objective(model, max_days)
    # model = add_profit_as_first_objective(model, J, L, data["jobs"])
    
    model, profit, max_days, max_jobs = add_mono_objective(model, profit, max_days, max_jobs)
    
    # maj du modèle
    model.update()
    
    # Paramétrage (mode mute)
    model.params.outputflag = 0

    return model, profit, max_days, max_jobs

In [44]:
def main(dataset):
    data = load_data(dataset)
    n_staff, horizon, n_qualifs, n_jobs = get_dims(data)
    points = []
    for i in tqdm(range(horizon, -1, -1)):
        for j in range(n_jobs, -1, -1):
            print(j, " / ", n_jobs)
            model, profit, max_days, max_jobs = prepare_model(data)
            model.addConstr(max_jobs <= j)
            model.addConstr(max_days <= i)
            model.update()
            model.optimize()
            point = np.array([round(i), round(j), round(profit.X)])
            points.append(point)
    points = np.array(points)
    return points

    
    

In [45]:
dataset = DATASET.MEDIUM
data = load_data(dataset)
n_staff, horizon, n_qualifs, n_jobs = get_dims(data)
points = main(dataset)
print(points)

  0%|          | 0/6 [00:00<?, ?it/s]

5  /  5
4  /  5
3  /  5
2  /  5


 17%|█▋        | 1/6 [00:00<00:03,  1.46it/s]

1  /  5
0  /  5
5  /  5
4  /  5
3  /  5
2  /  5


 33%|███▎      | 2/6 [00:01<00:02,  1.54it/s]

...

0  /  5
5  /  5
4  /  5
3  /  5
2  /  5
1  /  5


 83%|████████▎ | 5/6 [00:03<00:00,  1.44it/s]

0  /  5
5  /  5
4  /  5
3  /  5
2  /  5
1  /  5
0  /  5


100%|██████████| 6/6 [00:03<00:00,  1.64it/s]

[[ 5  5 70]
 [ 5  4 70]
 [ 5  3 70]
 [ 5  2 67]
 [ 5  1 45]
 [ 5  0  0]
 [ 4  5 70]
 [ 4  4 70]
 [ 4  3 70]
 [ 4  2 67]
 [ 4  1 45]
 [ 4  0  0]
 [ 3  5 70]
 [ 3  4 70]
 [ 3  3 70]
 [ 3  2 65]
 [ 3  1 42]
 [ 3  0  0]
 [ 2  5 70]
 [ 2  4 70]
 [ 2  3 70]
 [ 2  2 55]
 [ 2  1 30]
 [ 2  0  0]
 [ 1  5 65]
 [ 1  4 65]
 [ 1  3 55]
 [ 1  2 40]
 [ 1  1 20]
 [ 1  0  0]
 [ 0  5  0]
 [ 0  4  0]
 [ 0  3  0]
 [ 0  2  0]
 [ 0  1  0]
 [ 0  0  0]]


# Visualisation

In [46]:
print(points[:,0])

[5 5 5 5 5 5 4 4 4 4 4 4 3 3 3 3 3 3 2 2 2 2 2 2 1 1 1 1 1 1 0 0 0 0 0 0]


In [48]:
Z = np.array(points[:,2]).reshape(horizon+1,n_jobs+1)

X=np.linspace(horizon,0,horizon +1)
Y = np.linspace(n_jobs,0,n_jobs +1)
X, Y = np.meshgrid(X,Y)

print(X)
print(Y)
print(Z)


fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
fig.update_layout(scene=dict(xaxis_title="Days", yaxis_title="Jobs", zaxis_title="Benefit"))
fig.show()

[[5. 4. 3. 2. 1. 0.]
 [5. 4. 3. 2. 1. 0.]
 [5. 4. 3. 2. 1. 0.]
 [5. 4. 3. 2. 1. 0.]
 [5. 4. 3. 2. 1. 0.]
 [5. 4. 3. 2. 1. 0.]]
[[5. 5. 5. 5. 5. 5.]
 [4. 4. 4. 4. 4. 4.]
 [3. 3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2. 2.]
 [1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0.]]
[[70 70 70 67 45  0]
 [70 70 70 67 45  0]
 [70 70 70 65 42  0]
 [70 70 70 55 30  0]
 [65 65 55 40 20  0]
 [ 0  0  0  0  0  0]]
